In [2]:
# !pip install panda
# import random
# import pandas as pd
import dataGenerator
# import numpy as np

In [3]:
feature,label = dataGenerator.load_data()
sampleFeature,sampleLabel = dataGenerator.resampling(feature,label,1)
print sampleFeature.shape

(984, 30)


In [ ]:
# print(data.values)
print(len(data.columns))
# X = data.iloc[:,]
numIndex = len(data.columns)
featureVec = data.iloc[:,:numIndex-1]
labelVec = data.iloc[:,numIndex-1]
# print(featureVec.values[4,:])
# print(len(labelVec.values))
label = labelVec.values
label_0 = len(label[label == 1]) 
print label_0